KHỞI TẠO THƯ VIỆN

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from scipy.stats import randint


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


ĐỌC FILE

In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/clean_feature.csv")


In [ ]:
df1 = df1.drop_duplicates()       # Loại bỏ các trùng lặp
df1.shape

(443011, 8)

THÊM CỘT 'DATETIME'

In [ ]:
df1['datetime'] = pd.to_datetime(df1['Day'] + ' ' + df1['hour'].astype(str) + ':00:00')
df1.sort_values(by='datetime', inplace=True)
df1.reset_index(drop=True, inplace=True)


In [ ]:
df2 = df1.copy()

In [ ]:
df2.drop(labels='Day', axis=1, inplace=False)


,DoWeek,hour,count,passed,period,data,ServiceID,datetime
0,2,17,2974,2974,63.952589,612.081036,2,2021-12-29 17:00:00
1,2,17,894,894,63.000000,600.862416,7,2021-12-29 17:00:00
2,2,17,18,18,62.000000,14053.000000,5,2021-12-29 17:00:00
3,2,17,3244,3244,63.000000,955.297472,11,2021-12-29 17:00:00
4,2,17,2041,2041,63.938266,607.177854,2,2021-12-29 17:00:00
...,...,...,...,...,...,...,...,...
443006,4,18,2,2,62.000000,462.500000,1,2023-08-04 18:00:00
443007,4,18,59,59,62.000000,945.881356,5,2023-08-04 18:00:00
443008,4,18,2091,2091,63.000000,599.084648,7,2023-08-04 18:00:00
443009,4,18,7,7,0.000000,378358.142857,10,2023-08-04 18:00:00


THỰC HIỆN GROUPBY

In [ ]:
df3 = df2.groupby(['datetime', 'ServiceID', 'hour', 'DoWeek']).agg({
    'data': 'mean',
    'period': 'max',
    'count': 'sum',
    'passed': 'sum'
}).reset_index()


In [ ]:
# Thêm cột 'error' vào DataFrame
df3['error'] = np.where(
    df3['count'] != 0,
    np.abs(df3['count'] - df3['passed']) / df3['count'],
    0  # Nếu count = 0, gán error = 0
)
df3.head(10)

,datetime,ServiceID,hour,DoWeek,data,period,count,passed,error
0,2021-12-29 17:00:00,2,17,2,612.824589,64.00000,6550,6550,0.0
1,2021-12-29 17:00:00,5,17,2,11599.654840,62.00000,52,52,0.0
2,2021-12-29 17:00:00,7,17,2,601.058268,63.00000,2736,2736,0.0
3,2021-12-29 17:00:00,11,17,2,959.758965,63.00000,10253,10253,0.0
4,2021-12-29 18:00:00,2,18,2,613.510406,63.94213,34044,34044,0.0
5,2021-12-29 18:00:00,5,18,2,2037.468543,62.00000,444,444,0.0
6,2021-12-29 18:00:00,7,18,2,600.678181,63.00000,13853,13853,0.0
7,2021-12-29 18:00:00,11,18,2,946.017958,63.00000,52502,52502,0.0
8,2021-12-29 19:00:00,2,19,2,617.514337,64.00000,33444,33444,0.0
9,2021-12-29 19:00:00,5,19,2,2084.242552,62.00000,701,701,0.0


In [ ]:
df3['check'] = np.where(df3['passed'] < df3['count'], 1, 0)
df3['check_label'] = df3['check']

df3.head(10)


,datetime,ServiceID,hour,DoWeek,data,period,count,passed,error,check,check_label
0,2021-12-29 17:00:00,2,17,2,612.824589,64.00000,6550,6550,0.0,0,0
1,2021-12-29 17:00:00,5,17,2,11599.654840,62.00000,52,52,0.0,0,0
2,2021-12-29 17:00:00,7,17,2,601.058268,63.00000,2736,2736,0.0,0,0
3,2021-12-29 17:00:00,11,17,2,959.758965,63.00000,10253,10253,0.0,0,0
4,2021-12-29 18:00:00,2,18,2,613.510406,63.94213,34044,34044,0.0,0,0
5,2021-12-29 18:00:00,5,18,2,2037.468543,62.00000,444,444,0.0,0,0
6,2021-12-29 18:00:00,7,18,2,600.678181,63.00000,13853,13853,0.0,0,0
7,2021-12-29 18:00:00,11,18,2,946.017958,63.00000,52502,52502,0.0,0,0
8,2021-12-29 19:00:00,2,19,2,617.514337,64.00000,33444,33444,0.0,0,0
9,2021-12-29 19:00:00,5,19,2,2084.242552,62.00000,701,701,0.0,0,0


In [ ]:
# tạo số index
df3_reset = df3.reset_index()
df3_reset

,index,datetime,ServiceID,hour,DoWeek,data,period,count,passed,error,check,check_label
0,0,2021-12-29 17:00:00,2,17,2,612.824589,64.00000,6550,6550,0.0,0,0
1,1,2021-12-29 17:00:00,5,17,2,11599.654840,62.00000,52,52,0.0,0,0
2,2,2021-12-29 17:00:00,7,17,2,601.058268,63.00000,2736,2736,0.0,0,0
3,3,2021-12-29 17:00:00,11,17,2,959.758965,63.00000,10253,10253,0.0,0,0
4,4,2021-12-29 18:00:00,2,18,2,613.510406,63.94213,34044,34044,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
83517,83517,2023-08-04 18:00:00,5,18,4,1628.701794,62.00000,331,331,0.0,0,0
83518,83518,2023-08-04 18:00:00,7,18,4,599.175368,63.00000,9501,9501,0.0,0,0
83519,83519,2023-08-04 18:00:00,8,18,4,1330.250000,62.00000,3,3,0.0,0,0
83520,83520,2023-08-04 18:00:00,10,18,4,378357.702381,0.00000,33,33,0.0,0,0


CHUẨN BỊ DF CHO POSITIONAL ENCODING

In [ ]:
# Lấy các cột cần thiết và loại bỏ trùng lặp
subset_df = df3_reset[['datetime', 'DoWeek', 'hour']].drop_duplicates().reset_index(drop=True)

# Hiển thị kết quả
print(subset_df)


                 datetime  DoWeek  hour
0     2021-12-29 17:00:00       2    17
1     2021-12-29 18:00:00       2    18
2     2021-12-29 19:00:00       2    19
3     2021-12-29 20:00:00       2    20
4     2021-12-29 21:00:00       2    21
...                   ...     ...   ...
13984 2023-08-04 14:00:00       4    14
13985 2023-08-04 15:00:00       4    15
13986 2023-08-04 16:00:00       4    16
13987 2023-08-04 17:00:00       4    17
13988 2023-08-04 18:00:00       4    18

[13989 rows x 3 columns]


In [ ]:
# Tạo DataFrame ServiceID từ 0 đến 11
service_ids = pd.DataFrame({'ServiceID': range(12)})

# Kết hợp DataFrame ban đầu với ServiceID
expanded_df = subset_df.merge(service_ids, how='cross')


In [ ]:
expanded_df

,datetime,DoWeek,hour,ServiceID
0,2021-12-29 17:00:00,2,17,0
1,2021-12-29 17:00:00,2,17,1
2,2021-12-29 17:00:00,2,17,2
3,2021-12-29 17:00:00,2,17,3
4,2021-12-29 17:00:00,2,17,4
...,...,...,...,...
167863,2023-08-04 18:00:00,4,18,7
167864,2023-08-04 18:00:00,4,18,8
167865,2023-08-04 18:00:00,4,18,9
167866,2023-08-04 18:00:00,4,18,10


In [ ]:
from google.colab import files

# Lưu DataFrame expanded_df thành file CSV
expanded_df.to_csv("datetime_ServiceID.csv", index=False)

# Tải file về máy
files.download("datetime_ServiceID.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def positional_encoding_direct_hour(hour, d=12, c=1e-3):
    encoding = []
    for i in range(int(d / 2)):
        encoding.append(np.sin(hour * (c ** (4 * i / d))))
        encoding.append(np.cos(hour * (c ** (4 * i / d))))
    return encoding

def positional_encoding_direct_DoWeek(day_of_week, d=12, c=1e-4):
    encoding = []
    for i in range(int(d / 2)):
        encoding.append(np.sin(day_of_week * (c ** (4 * i / d))))
        encoding.append(np.cos(day_of_week * (c ** (4 * i / d))))
    return encoding



In [ ]:
subset_df['hour_encoding'] = subset_df['hour'].apply(positional_encoding_direct_hour)
subset_df['DoWeek_encoding'] = subset_df['DoWeek'].apply(positional_encoding_direct_DoWeek)


In [ ]:
# Mở rộng cột hour_encoding thành các cột riêng
hour_encoding_df = pd.DataFrame(
    subset_df['hour_encoding'].to_list(),
    columns=[f'hour_encoding_{i}' for i in range(len(subset_df['hour_encoding'].iloc[0]))]
)

# Mở rộng cột DoWeek_encoding thành các cột riêng
doweek_encoding_df = pd.DataFrame(
    subset_df['DoWeek_encoding'].to_list(),
    columns=[f'DoWeek_encoding_{i}' for i in range(len(subset_df['DoWeek_encoding'].iloc[0]))]
)

# Kết hợp lại với subset_df
subset_df = pd.concat([subset_df.drop(['hour_encoding', 'DoWeek_encoding'], axis=1), hour_encoding_df, doweek_encoding_df], axis=1)

# Hiển thị kết quả
subset_df.head(50)


,datetime,DoWeek,hour,hour_encoding_0,hour_encoding_1,hour_encoding_2,hour_encoding_3,hour_encoding_4,hour_encoding_5,hour_encoding_6,...,DoWeek_encoding_2,DoWeek_encoding_3,DoWeek_encoding_4,DoWeek_encoding_5,DoWeek_encoding_6,DoWeek_encoding_7,DoWeek_encoding_8,DoWeek_encoding_9,DoWeek_encoding_10,DoWeek_encoding_11
0,2021-12-29 17:00:00,2,17,-0.961397,-0.275163,0.991665,-0.128844,0.169182,0.985585,0.016999,...,0.092699,0.995694,0.004309,0.999991,0.0002,1.0,0.000009,1.0,4.308869e-07,1.0
1,2021-12-29 18:00:00,2,18,-0.750987,0.660317,0.973848,-0.227202,0.179030,0.983844,0.017999,...,0.092699,0.995694,0.004309,0.999991,0.0002,1.0,0.000009,1.0,4.308869e-07,1.0
2,2021-12-29 19:00:00,2,19,0.149877,0.988705,0.946300,-0.323290,0.188859,0.982004,0.018999,...,0.092699,0.995694,0.004309,0.999991,0.0002,1.0,0.000009,1.0,4.308869e-07,1.0
3,2021-12-29 20:00:00,2,20,0.912945,0.408082,0.909297,-0.416147,0.198669,0.980067,0.019999,...,0.092699,0.995694,0.004309,0.999991,0.0002,1.0,0.000009,1.0,4.308869e-07,1.0
4,2021-12-29 21:00:00,2,21,0.836656,-0.547729,0.863209,-0.504846,0.208460,0.978031,0.020998,...,0.092699,0.995694,0.004309,0.999991,0.0002,1.0,0.000009,1.0,4.308869e-07,1.0
5,2021-12-29 22:00:00,2,22,-0.008851,-0.999961,0.808496,-0.588501,0.218230,0.975897,0.021998,...,0.092699,0.995694,0.004309,0.999991,0.0002,1.0,0.000009,1.0,4.308869e-07,1.0
6,2021-12-29 23:00:00,2,23,-0.846220,-0.532833,0.745705,-0.666276,0.227978,0.973666,0.022998,...,0.092699,0.995694,0.004309,0.999991,0.0002,1.0,0.000009,1.0,4.308869e-07,1.0
7,2021-12-30 00:00:00,3,0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.138798,0.990321,0.006463,0.999979,0.0003,1.0,0.000014,1.0,6.463304e-07,1.0
8,2021-12-30 01:00:00,3,1,0.841471,0.540302,0.099833,0.995004,0.010000,0.999950,0.001000,...,0.138798,0.990321,0.006463,0.999979,0.0003,1.0,0.000014,1.0,6.463304e-07,1.0
9,2021-12-30 02:00:00,3,2,0.909297,-0.416147,0.198669,0.980067,0.019999,0.999800,0.002000,...,0.138798,0.990321,0.006463,0.999979,0.0003,1.0,0.000014,1.0,6.463304e-07,1.0


MINMAXSCALE DỮ LIỆU VÀ THỰC HIỆN PIVOT

In [ ]:

df_scaled = df3_reset.copy()

# Xác định các cột cần scale
columns_to_scale = ['data', 'period', 'passed', 'count']

# Áp dụng MinMaxScaler
scaler = MinMaxScaler()
df_scaled[columns_to_scale] = scaler.fit_transform(df_scaled[columns_to_scale])

# Sau khi scale, thực hiện pivot
df_feature = df_scaled.pivot(index='datetime', columns='ServiceID', values=['data', 'period', 'passed', 'error', 'check'])

# Đổi tên các cột
df_feature.columns = [f'{col[0]}_{col[1]}' for col in df_feature.columns]

df_feature = df_feature.fillna(-1)

# Reset lại chỉ số nếu cần
df_feature = df_feature.reset_index()

print(df_feature)


                 datetime    data_0    data_1    data_2    data_3    data_4  \
0     2021-12-29 17:00:00 -1.000000 -1.000000  0.001620 -1.000000 -1.000000   
1     2021-12-29 18:00:00 -1.000000 -1.000000  0.001622 -1.000000 -1.000000   
2     2021-12-29 19:00:00 -1.000000 -1.000000  0.001632 -1.000000 -1.000000   
3     2021-12-29 20:00:00 -1.000000 -1.000000  0.001604 -1.000000 -1.000000   
4     2021-12-29 21:00:00  0.002498 -1.000000  0.001601  0.003880 -1.000000   
...                   ...       ...       ...       ...       ...       ...   
13984 2023-08-04 14:00:00  0.003610  0.001507  0.001750 -1.000000  0.071668   
13985 2023-08-04 15:00:00  0.004744  0.001486  0.001736  0.002169  0.116999   
13986 2023-08-04 16:00:00  0.004051  0.001543  0.001743 -1.000000  0.069379   
13987 2023-08-04 17:00:00  0.003326  0.001506  0.001702  0.003880  0.062320   
13988 2023-08-04 18:00:00  0.002498  0.001357  0.001709 -1.000000  0.197573   

         data_5    data_6    data_7    data_8  ... 

GHÉP CÁC FEATURE POSITIONAL ENCODING VÀ FEATURE  ĐÃ PIVOT

In [ ]:
# Ghép hai DataFrame dựa trên cột 'datetime'
merged_df = pd.merge(subset_df, df_feature, on='datetime', how='inner').drop(['DoWeek', 'hour'], axis=1)

# Hiển thị kết quả
print(merged_df)

                 datetime  hour_encoding_0  hour_encoding_1  hour_encoding_2  \
0     2021-12-29 17:00:00        -0.961397        -0.275163         0.991665   
1     2021-12-29 18:00:00        -0.750987         0.660317         0.973848   
2     2021-12-29 19:00:00         0.149877         0.988705         0.946300   
3     2021-12-29 20:00:00         0.912945         0.408082         0.909297   
4     2021-12-29 21:00:00         0.836656        -0.547729         0.863209   
...                   ...              ...              ...              ...   
13984 2023-08-04 14:00:00         0.990607         0.136737         0.985450   
13985 2023-08-04 15:00:00         0.650288        -0.759688         0.997495   
13986 2023-08-04 16:00:00        -0.287903        -0.957659         0.999574   
13987 2023-08-04 17:00:00        -0.961397        -0.275163         0.991665   
13988 2023-08-04 18:00:00        -0.750987         0.660317         0.973848   

       hour_encoding_3  hour_encoding_4

TẠO DF ĐỂ CHUẨN BỊ TRÍCH XUẤT LABEL

In [ ]:

# Pivot dữ liệu để chuyển 'ServiceID' thành các cột
df_label = df3_reset.pivot(index='datetime', columns='ServiceID', values=['check_label'])

# Đổi tên các cột để phù hợp định dạng 'data_0', 'period_0', ...
df_label.columns = [f'{col[0]}_{col[1]}' for col in df_label.columns]
df_label = df_label.fillna(1)
# Reset lại chỉ số nếu cần
df_label = df_label.reset_index()
for col in df_label.columns[1:]:  # Bỏ qua cột 'datetime'
    df_label[col] = df_label[col].astype('category')
print(df_label)


                 datetime check_label_0 check_label_1 check_label_2  \
0     2021-12-29 17:00:00           1.0           1.0           0.0   
1     2021-12-29 18:00:00           1.0           1.0           0.0   
2     2021-12-29 19:00:00           1.0           1.0           0.0   
3     2021-12-29 20:00:00           1.0           1.0           0.0   
4     2021-12-29 21:00:00           0.0           1.0           0.0   
...                   ...           ...           ...           ...   
13984 2023-08-04 14:00:00           0.0           0.0           0.0   
13985 2023-08-04 15:00:00           0.0           0.0           0.0   
13986 2023-08-04 16:00:00           0.0           0.0           0.0   
13987 2023-08-04 17:00:00           0.0           0.0           0.0   
13988 2023-08-04 18:00:00           0.0           0.0           0.0   

      check_label_3 check_label_4 check_label_5 check_label_6 check_label_7  \
0               1.0           1.0           0.0           1.0       

LẤY OBSERVATION TRONG 2 GIỜ TRƯỚC T

In [ ]:
def filter_data_last_2_hours(data_in, t):
    # Kiểm tra kiểu dữ liệu cột 'datetime' để đảm bảo là kiểu datetime
    if not pd.api.types.is_datetime64_any_dtype(data_in['datetime']):
        raise ValueError("Column 'datetime' must be of datetime dtype.")

    # Tính toán thời gian bắt đầu và kết thúc của khoảng 2 giờ trước t
    time_window_start = t - pd.Timedelta(hours=2)
    time_window_end = t

    # Lọc dữ liệu trong khoảng thời gian 2 giờ trước t
    filtered_data = data_in[(data_in['datetime'] >= time_window_start) & (data_in['datetime'] <= time_window_end)]

    # Kiểm tra nếu không đủ 3 hàng hoặc chứa giá trị NaN
    if filtered_data.shape[0] != 3 or filtered_data.isnull().values.any():
        print(f"Invalid data for the last 2 hours of {t}. Skipping this time window.")
        return None

    # Xóa cột datetime
    filtered_data = filtered_data.drop(columns='datetime')
    # Chuyển hàng thành cột, cột thành hàng
    filtered_data = filtered_data.T
    # Trả về dữ liệu đã lọc
    return filtered_data


In [ ]:
# Giả sử t là thời điểm cụ thể mà bạn muốn lọc dữ liệu
t = pd.Timestamp('2021-12-31 11:00:00')  # Thay thế bằng thời điểm bạn muốn

# Lọc dữ liệu trong 2 giờ trước t
filtered_data = filter_data_last_2_hours(merged_df, t)
filtered_data
filtered_data2 = filtered_data.to_numpy().reshape(7, 12, 3)


In [ ]:
filtered_data2

array([[[ 4.12118485e-01, -5.44021111e-01, -9.99990207e-01],
        [-9.11130262e-01, -8.39071529e-01,  4.42569799e-03],
        [ 7.83326910e-01,  8.41470985e-01,  8.91207360e-01],
        [ 6.21609968e-01,  5.40302306e-01,  4.53596121e-01],
        [ 8.98785492e-02,  9.98334166e-02,  1.09778301e-01],
        [ 9.95952733e-01,  9.95004165e-01,  9.93956098e-01],
        [ 8.99987850e-03,  9.99983333e-03,  1.09997782e-02],
        [ 9.99959500e-01,  9.99950000e-01,  9.99939501e-01],
        [ 8.99999879e-04,  9.99999833e-04,  1.09999978e-03],
        [ 9.99999595e-01,  9.99999500e-01,  9.99999395e-01],
        [ 8.99999999e-05,  9.99999998e-05,  1.10000000e-04],
        [ 9.99999996e-01,  9.99999995e-01,  9.99999994e-01]],

       [[-7.56802495e-01, -7.56802495e-01, -7.56802495e-01],
        [-6.53643621e-01, -6.53643621e-01, -6.53643621e-01],
        [ 1.84598724e-01,  1.84598724e-01,  1.84598724e-01],
        [ 9.82813976e-01,  9.82813976e-01,  9.82813976e-01],
        [ 8.61763209e-

LẤY LABEL TRONG 1 GIỜ SAU T

In [ ]:
def get_labels(data_in, t):
    # Tính toán thời gian t + 1
    time_plus_one = t + pd.Timedelta(hours=1)

    # Lọc nhãn tại thời điểm t + 1
    labels = data_in[data_in['datetime'] == time_plus_one][['check_label_0', 'check_label_1', 'check_label_2', 'check_label_3',
                                                             'check_label_4', 'check_label_5', 'check_label_6', 'check_label_7',
                                                             'check_label_8', 'check_label_9', 'check_label_10', 'check_label_11']].astype(int)

    # Kiểm tra nếu labels rỗng hoặc chứa giá trị NaN, bỏ qua
    if labels.empty or labels.isnull().values.any():
        print(f"No valid labels found for time {time_plus_one}. Skipping this time window.")
        return None  # Trả về None

    return labels


TẠO TẬP TRAIN VÀ TEST

In [ ]:
# Tính ngày giới hạn cho 1.5 năm
start_date = df3_reset['datetime'].min()
train_limit_date = start_date + pd.Timedelta(days=365 * 1)

# Chia tập train và test
train_data = df3_reset[df3_reset['datetime'] <= train_limit_date]
test_data = df3_reset[df3_reset['datetime'] > train_limit_date]

In [ ]:
# Tạo danh sách tất cả các thời điểm duy nhất trong cột datetime
unique_datetimes_f = train_data['datetime'].unique()
unique_datetimes_f = unique_datetimes_f[2:-1]  # Loại bỏ 2 phần tử đầu và 1 phần tử cuối
unique_datetimes_f

<DatetimeArray>
['2021-12-29 19:00:00', '2021-12-29 20:00:00', '2021-12-29 21:00:00',
 '2021-12-29 22:00:00', '2021-12-29 23:00:00', '2021-12-30 00:00:00',
 '2021-12-30 01:00:00', '2021-12-30 02:00:00', '2021-12-30 03:00:00',
 '2021-12-30 04:00:00',
 ...
 '2022-12-29 07:00:00', '2022-12-29 08:00:00', '2022-12-29 09:00:00',
 '2022-12-29 10:00:00', '2022-12-29 11:00:00', '2022-12-29 12:00:00',
 '2022-12-29 13:00:00', '2022-12-29 14:00:00', '2022-12-29 15:00:00',
 '2022-12-29 16:00:00']
Length: 8753, dtype: datetime64[ns]

In [ ]:
# Tạo danh sách tất cả các thời điểm duy nhất trong cột datetime
unique_datetimes_t = test_data['datetime'].unique()
unique_datetimes_t = unique_datetimes_t[2:-1]  # Loại bỏ 2 phần tử đầu và 1 phần tử cuối
unique_datetimes_t

<DatetimeArray>
['2022-12-29 20:00:00', '2022-12-29 21:00:00', '2022-12-29 22:00:00',
 '2022-12-29 23:00:00', '2022-12-30 00:00:00', '2022-12-30 01:00:00',
 '2022-12-30 02:00:00', '2022-12-30 03:00:00', '2022-12-30 04:00:00',
 '2022-12-30 05:00:00',
 ...
 '2023-08-04 08:00:00', '2023-08-04 09:00:00', '2023-08-04 10:00:00',
 '2023-08-04 11:00:00', '2023-08-04 12:00:00', '2023-08-04 13:00:00',
 '2023-08-04 14:00:00', '2023-08-04 15:00:00', '2023-08-04 16:00:00',
 '2023-08-04 17:00:00']
Length: 5230, dtype: datetime64[ns]

TẠO X_TRAIN, Y_TRAIN, X_TEST, Y_TEST

In [ ]:
train_data_features = []
train_data_labels = []
test_data_features = []
test_data_labels = []

for t in unique_datetimes_f:
    # Lọc dữ liệu cho 2 giờ trước thời điểm t
    features_data = filter_data_last_2_hours(merged_df, t)
    # Lấy dữ liệu 1 giờ sau
    labels_data = get_labels(df_label, t)

    # Kiểm tra nếu features_data hoặc labels_data là None
    if features_data is None or labels_data is None:
        continue  # Bỏ qua vòng lặp này

    # Nếu dữ liệu hợp lệ, thêm vào danh sách
    train_data_features.append(features_data)
    train_data_labels.append(labels_data)

for t in unique_datetimes_t:
    # Lọc dữ liệu cho 2 giờ trước thời điểm t
    features_data = filter_data_last_2_hours(merged_df, t)
    # Lấy dữ liệu 1 giờ sau
    labels_data = get_labels(df_label, t)

    # Kiểm tra nếu features_data hoặc labels_data là None
    if features_data is None or labels_data is None:
        continue  # Bỏ qua vòng lặp này

    # Nếu dữ liệu hợp lệ, thêm vào danh sách
    test_data_features.append(features_data)
    test_data_labels.append(labels_data)




No valid labels found for time 2022-01-09 00:00:00. Skipping this time window.
Invalid data for the last 2 hours of 2022-01-09 03:00:00. Skipping this time window.
Invalid data for the last 2 hours of 2022-01-09 04:00:00. Skipping this time window.
No valid labels found for time 2022-05-03 02:00:00. Skipping this time window.
Invalid data for the last 2 hours of 2022-05-03 03:00:00. Skipping this time window.
Invalid data for the last 2 hours of 2022-05-03 04:00:00. Skipping this time window.
No valid labels found for time 2022-08-27 01:00:00. Skipping this time window.
Invalid data for the last 2 hours of 2022-08-27 02:00:00. Skipping this time window.
Invalid data for the last 2 hours of 2022-08-27 03:00:00. Skipping this time window.


In [ ]:
# Kết quả: Gộp tất cả các DataFrame đã lọc lại thành một DataFrame
X_train = np.stack(train_data_features, axis=0)
Y_train = np.concatenate(train_data_labels, axis=0)

X_test = np.stack(test_data_features, axis=0)
Y_test = np.concatenate(test_data_labels, axis=0)

In [ ]:
X_test.shape

(5230, 84, 3)

XÂY DỰNG MODEL CONV1D

In [ ]:
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, MaxPooling1D, Dropout, Dense, LeakyReLU, Flatten, Concatenate

# Hàm Conv Block
def conv_block(x, filters, kernel_size=2):
    x = Conv1D(filters, kernel_size, activation='relu', padding='same')(x)
    return x

# Hàm tính F1 score
def f1_score_metric(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32) # Cast y_true to float32
    y_pred = tf.round(y_pred)  # Chuyển xác suất về nhị phân 0 hoặc 1
    tp = tf.reduce_sum(tf.cast(y_true * y_pred, 'float'), axis=0)  # dự đoán đúng mãu 1
    fp = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, 'float'), axis=0)  # dự đoán sai các mãu 1 là 0
    fn = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), 'float'), axis=0)  # dự đoán sai các mẫu 0 là 1

    # Tính precision và recall
    precision = tp / (tp + fp + tf.keras.backend.epsilon()) #  Tỉ lệ mẫu được dự đoán đúng trên tổng số mẫu được dự đoán là 1.
    recall = tp / (tp + fn + tf.keras.backend.epsilon())    #  Tỉ lệ mẫu được dự đoán đúng trên tổng số mẫu thực tế là 1.

    # Tính F1 score
    f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    # Kiểm tra trường hợp y_true chỉ có 0 hoặc 1
    total_elements = tf.reduce_sum(y_true, axis=0) + tf.reduce_sum(1 - y_true, axis=0)
    single_class_mask = tf.logical_or(tf.reduce_sum(y_true, axis=0) == total_elements,
                                      tf.reduce_sum(y_true, axis=0) == 0)
    f1 = tf.where(single_class_mask, tf.ones_like(f1), f1)  # Đặt F1 score bằng 1 cho nhãn đơn lớp

    return tf.reduce_mean(f1)

# Định nghĩa mô hình
def build_model():
    input_layer = Input(shape=(84, 3))

    # Tách các phần của đầu vào
    input_hour = input_layer[:, 0:12, :]
    input_DoWeek = input_layer[:, 12:24, :]
    input_data = input_layer[:, 24:36, :]
    input_period = input_layer[:, 36:48, :]
    input_passed = input_layer[:, 48:60, :]
    input_error = input_layer[:, 60:72, :]
    input_check = input_layer[:, 72:84, :]

    # Áp dụng Conv Block cho các phần có tính chất chuỗi thời gian
    conv_hour = conv_block(input_hour, 16)
    conv_DoWeek = conv_block(input_DoWeek, 16)
    conv_passed = conv_block(input_passed, 16)
    conv_period = conv_block(input_period, 16)
    conv_data = conv_block(input_data, 16)
    conv_fail = conv_block(input_error, 16)
    conv_check = conv_block(input_check, 16)

    # Ghép nối các khối Conv lại với nhau
    merged_conv = Concatenate(axis=-1)([conv_hour, conv_DoWeek, conv_passed, conv_period, conv_data, conv_fail, conv_check])

    # Các lớp Conv1D sau khi ghép nối
    x = Conv1D(64, 3, activation='relu')(merged_conv)
    x = Conv1D(32, 3, activation='relu')(x)

    # Flatten và các lớp Dense
    combined = Flatten()(x)
    combined = Dense(48, activation='relu')(combined)  # Giảm số lượng nơ-ron
    combined = Dense(24, activation='relu')(combined)  # Giảm số lượng nơ-ron

    # Lớp đầu ra
    output_layer = Dense(12, activation='sigmoid')(combined)

    # Khởi tạo mô hình
    model = Model(inputs=input_layer, outputs=output_layer)

    # Biên dịch mô hình
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_score_metric])

    return model

# Xây dựng mô hình
model = build_model()

# Xem tóm tắt mô hình
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 84, 3)]              0         []                            
                                                                                                  
 tf.__operators__.getitem (  (None, 12, 3)                0         ['input_1[0][0]']             
 SlicingOpLambda)                                                                                 
                                                                                                  
 tf.__operators__.getitem_1  (None, 12, 3)                0         ['input_1[0][0]']             
  (SlicingOpLambda)                                                                               
                                                                                              

TRAIN MODEL

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# X_train, X_test, Y_train, Y_test

# Khởi tạo và biên dịch mô hình
model = build_model()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[f1_score_metric])

# Các callback
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-7),
    ModelCheckpoint('best_model_test.keras', monitor='val_f1_score_metric', save_best_only=True, mode='max')
]

# Huấn luyện mô hình
history = model.fit(X_train, Y_train, epochs=100, validation_data=(X_test, Y_test), callbacks=callbacks)

# Tìm giá trị F1 score lớn nhất
max_f1_score = np.max(history.history['val_f1_score_metric'])
print(f"Max F1 Score: {max_f1_score:.4f}")

# Để tải trọng số tốt nhất đã lưu
model.load_weights('best_model_test.keras')

# Đánh giá mô hình trên dữ liệu kiểm tra
_, test_f1_score = model.evaluate(X_test, Y_test, verbose=0)
print(f"Test F1 Score: {test_f1_score:.4f}")


Epoch 1/100
274/274 [==============================] - 4s 7ms/step - loss: 0.1379 - f1_score_metric: 0.9060 - val_loss: 0.4270 - val_f1_score_metric: 0.9077 - lr: 0.0050
Epoch 2/100
274/274 [==============================] - 1s 5ms/step - loss: 0.1095 - f1_score_metric: 0.9312 - val_loss: 0.4051 - val_f1_score_metric: 0.9064 - lr: 0.0050
Epoch 3/100
274/274 [==============================] - 1s 5ms/step - loss: 0.1066 - f1_score_metric: 0.9322 - val_loss: 0.5338 - val_f1_score_metric: 0.9032 - lr: 0.0050
Epoch 4/100
274/274 [==============================] - 2s 6ms/step - loss: 0.1050 - f1_score_metric: 0.9337 - val_loss: 0.5332 - val_f1_score_metric: 0.9104 - lr: 0.0050
Epoch 5/100
274/274 [==============================] - 2s 6ms/step - loss: 0.1039 - f1_score_metric: 0.9354 - val_loss: 0.5023 - val_f1_score_metric: 0.9111 - lr: 0.0050
Epoch 6/100
274/274 [==============================] - 2s 5ms/step - loss: 0.1025 - f1_score_metric: 0.9352 - val_loss: 0.5053 - val_f1_score_metric: 

F1-RECALL, AUC, ACCURACY

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score, accuracy_score

# Dự đoán từ mô hình
y_pred = model.predict(X_test)

# Chuyển đổi xác suất thành nhãn (biến đổi kết quả phân loại)
y_pred_binary = (y_pred > 0.5).astype(int)

# Tính confusion matrix và báo cáo phân loại cho từng label từ 0 đến 11
f1_scores = []  # Lưu trữ F1 scores của từng label
accuracies = []  # Lưu trữ accuracy của từng label

for i in range(12):
    print(f"\nProcessing label {i}:")

    # Kiểm tra xem có dữ liệu cho label này hay không
    if np.sum(Y_test[:, i]) == 0:
        print(f"Label {i} không có dữ liệu trong Y_test. Hiển thị giá trị 0.\n")
        print(f"Confusion matrix for label {i}:\n[[0 0]\n [0 0]]")
        print(f"Classification Report for label {i}:\nprecision    recall  f1-score   support\n"
              f"     Class 0       0.0      0.0      0.0        0\n"
              f"     Class 1       0.0      0.0      0.0        0\n")
        print(f"AUC for label {i}: N/A\n")
        f1_scores.append(0)
        accuracies.append(0)
    else:
        # Tính Confusion Matrix
        cm = confusion_matrix(Y_test[:, i], y_pred_binary[:, i])
        print(f"Confusion matrix for label {i}:\n", cm)

        # In báo cáo phân loại với tham số zero_division=0
        report = classification_report(Y_test[:, i], y_pred_binary[:, i], target_names=["Class 0", "Class 1"], zero_division=0)
        print(f"Classification Report for label {i}:\n", report)

        # Tính AUC cho label này
        auc = roc_auc_score(Y_test[:, i], y_pred[:, i])  # Sử dụng xác suất (y_pred) thay vì nhãn nhị phân

        # Tính F1 Score và Accuracy cho từng label
        f1 = f1_score(Y_test[:, i], y_pred_binary[:, i])
        accuracy = accuracy_score(Y_test[:, i], y_pred_binary[:, i])

        f1_scores.append(f1)
        accuracies.append(accuracy)

    # In Average F1 Score và Average Accuracy cho từng label
    print(f"Average F1 Score for label {i}: {f1}")
    print(f"Average Accuracy for label {i}: {accuracy}")
    print(f"AUC for label {i}: {auc}\n")

164/164 [==============================] - 1s 2ms/step

Processing label 0:
Confusion matrix for label 0:
 [[1892  704]
 [ 331 2303]]
Classification Report for label 0:
               precision    recall  f1-score   support

     Class 0       0.85      0.73      0.79      2596
     Class 1       0.77      0.87      0.82      2634

    accuracy                           0.80      5230
   macro avg       0.81      0.80      0.80      5230
weighted avg       0.81      0.80      0.80      5230

Average F1 Score for label 0: 0.8165218932813331
Average Accuracy for label 0: 0.8021032504780115
AUC for label 0: 0.8773183409321976


Processing label 1:
Confusion matrix for label 1:
 [[2412 1146]
 [  65 1607]]
Classification Report for label 1:
               precision    recall  f1-score   support

     Class 0       0.97      0.68      0.80      3558
     Class 1       0.58      0.96      0.73      1672

    accuracy                           0.77      5230
   macro avg       0.78      0.82  

In [ ]:
# from google.colab import files  # Thư viện giúp tải file xuống từ Google Colab

# df_results.to_csv('model_f1_scores.csv', index=False)
# files.download('model_f1_scores.csv')


NameError: name 'df_results' is not defined

In [ ]:
import pandas as pd
from google.colab import files  # Thư viện giúp tải file xuống từ Google Colab

# Giả sử bạn đã có X_train, X_test, Y_train, Y_test và mô hình đã được huấn luyện

# Dự đoán trên cả tập huấn luyện và tập kiểm tra
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Chuyển đổi kết quả dự đoán sang dạng nhị phân (0 hoặc 1) nếu là phân loại nhị phân
y_pred_train_binary = (y_pred_train > 0.5).astype(int)
y_pred_test_binary = (y_pred_test > 0.5).astype(int)

# Tạo DataFrame cho cả tập huấn luyện và kiểm tra
train_results = pd.DataFrame({
    'Actual': Y_train.flatten(),  # Giá trị thực tế từ Y_train
    'Predicted': y_pred_train_binary.flatten(),  # Giá trị dự đoán từ mô hình
    'Dataset': ['Train'] * len(Y_train.flatten())  # Thay đổi len(Y_train) thành len(Y_train.flatten())
})

test_results = pd.DataFrame({
    'Actual': Y_test.flatten(),  # Giá trị thực tế từ Y_test
    'Predicted': y_pred_test_binary.flatten(),  # Giá trị dự đoán từ mô hình
    'Dataset': ['Test'] * len(Y_test.flatten())  # Thay đổi len(Y_test) thành len(Y_test.flatten())
})

# Kết hợp cả hai DataFrame (train và test)
all_results = pd.concat([train_results, test_results], ignore_index=True)

# Lưu kết quả vào file CSV
all_results.to_csv('model_predictions_all.csv', index=False)

# Tải file CSV xuống máy tính
files.download('model_predictions_all.csv')

print("Kết quả dự đoán đã được lưu vào file 'model_predictions_all.csv' và đang được tải xuống.")

164/164 [==============================] - 0s 2ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Kết quả dự đoán đã được lưu vào file 'model_predictions_all.csv' và đang được tải xuống.


In [ ]:
all_results.head()

,Actual,Predicted,Dataset
0,1,1,Train
1,1,1,Train
2,0,0,Train
3,1,1,Train
4,1,1,Train


In [ ]:
import pandas as pd

# Assuming you already have the 'all_results' DataFrame
num_rows = all_results.shape[0]

# Create a new column for 'serviceID'
all_results['serviceID'] = [(i // 12) + 1 for i in range(num_rows)]

# Reshape using pivot_table
pivot_df = all_results.pivot_table(
    index='serviceID',  # Group by serviceID
    columns=all_results.groupby('serviceID').cumcount(),  # Create columns for each row within a serviceID group
    values=['Actual', 'Predicted'],  # Values to include in the pivot table
    aggfunc='first'  # Use the first value for each cell (you can change this if needed)
).reset_index()

# Flatten MultiIndex columns for easier access
pivot_df.columns = ['_'.join(str(s) for s in col) for col in pivot_df.columns]

# Display the reshaped DataFrame
print(pivot_df)

       serviceID_  Actual_0  Actual_1  Actual_2  Actual_3  Actual_4  Actual_5  \
0               1         1         1         0         1         1         0   
1               2         0         1         0         1         1         0   
2               3         1         1         0         1         1         0   
3               4         0         1         0         1         1         0   
4               5         0         1         0         1         1         0   
...           ...       ...       ...       ...       ...       ...       ...   
13969       13970         0         0         0         1         0         0   
13970       13971         0         0         0         1         0         0   
13971       13972         0         0         0         1         0         0   
13972       13973         0         0         0         1         0         0   
13973       13974         0         0         0         1         0         0   

       Actual_6  Actual_7  